# Urban Henges with OSMNX

This was inspired by @Puntofisso's work colouring streetnames using OSMnx.

I wanted to use python and OMNX to create animations to show urban 'henges' - like ManhattanHenge. This code highlights where urban streets align with the angle of the rising or setting sun throughout the year. At the summer solstice, the sun rises in the North East; at the winter solstice it's close to the South East. By iterating through the possible angles, I created frames to assemble into a GIF. Streets are highlighted at the angle of the rising sun.

For an urban henge to really work, you would need a view of the horizon - quite challenging in an urban context. So these 'henges' are theoretical. You would also see the full effect of the sun when it's a couple of degrees above the horizon, so I've designed the highlight to start at the angle of the rising sun and grow for a couple of degrees before decaying.

OSMnx is an amazing package which allows you to download Open Streetmaps as networks of streets to manipulate in python with Networkx and Pandas. It's created and maintained by Geoff Boeing @gboeing
https://geoffboeing.com/2016/11/osmnx-python-street-networks/


In [1]:
import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import networkx as nx
import math
import matplotlib.pyplot as plt
from PIL import Image
import glob
import os
from datetime import datetime, timedelta
%matplotlib inline
ox.config(log_console=True, use_cache=True)



## Solar position equations

In [1]:

# declination function adapted from https://pvlib-python.readthedocs.io/en/stable/_modules/pvlib/solarposition.html
# other functions developed from formulae in http://mypages.iit.edu/~maslanka/SolarGeo.pdf and https://www.esrl.noaa.gov/gmd/grad/solcalc/solareqns.PDF

def declination(day_of_year):
    #returns declination in degrees
    offset = 1
    day_angle = (2. * np.pi / 365.) * (day_of_year - offset)

    return 57.29*(
        0.006918 -
        0.399912 * np.cos(day_angle) + 0.070257 * np.sin(day_angle) -
        0.006758 * np.cos(2. * day_angle) + 0.000907 * np.sin(2. * day_angle) -
        0.002697 * np.cos(3. * day_angle) + 0.00148 * np.sin(3. * day_angle))

    
def hour_angle(latitude, declination_angle):
    lat = np.radians(latitude)
    decl = np.radians(declination_angle)    
    cos_hour_angle = -np.tan(lat) * np.tan(decl)
    hour_angle = np.arccos(cos_hour_angle)
    return (57.29*hour_angle)
                   
def azimuth(latitude, hour_angle, declination_angle):
    lat = np.radians(latitude)
    decl = np.radians(declination_angle)
    ha = np.radians(hour_angle)
    alt = np.radians(0.833) # angle of altitude of sun at sunrise in degrees
    
    cos_azimuth = (np.sin(decl)*np.cos(lat)) - (np.cos(decl)*np.cos(ha)*np.sin(lat))/np.cos(alt)
    azimuth = np.arccos(cos_azimuth)
    return 57.29*azimuth

def sunrise_angle(latitude, day_of_year):

    sunrise_decl = declination(day_of_year)
    sunrise_angle = hour_angle(latitude, sunrise_decl)
    sunrise_azimuth = azimuth(latitude, sunrise_angle, sunrise_decl)
    return round(sunrise_azimuth, 1)
    

## OSMNX and plotting Functions

In [ ]:


def make_graph(place, which_result):
    # makes the graph structure; depending on the size/administrative level of the city, 
    # may need to change the 'which result' variable to 1 or 2
    G = ox.graph_from_place(place, network_type='all', which_result=which_result)
    return G
    
def make_df(G):
    # turns the graph into a df of streets
    df = ox.graph_to_gdfs(G, nodes=False)
    return df
    

def add_features(df, day_of_year):
    # adds some new features to the df to help with customising the plot
    lat = find_average_latitude(df)
    sunrise = sunrise_angle(lat, day_of_year)
    df['angle'] = df['geometry'].apply(lambda x: find_angle(x))
    df['colour'] = df['angle'].apply(lambda x: colour_fading(x, sunrise)['colour'])
    df['linewidth'] = df['angle'].apply(lambda x: colour_fading(x, sunrise)['linewidth'])
    df['alpha'] = df['angle'].apply(lambda x: colour_fading(x, sunrise)['alpha'])
    
    return df


def find_angle(street):
    if len(street.xy[0]) > 2:
        return 0
    else:
        X1, X2 = street.xy[0][0], street.xy[0][1]
        Y1, Y2 = street.xy[1][0], street.xy[1][1]
        x = X1 - X2
        y = Y1 - Y2
        if x == 0:
            return 0
        if y == 0:
            return 0
        else:
            return round(math.degrees(math.atan(x/y)), 2)

def find_average_latitude(df):
    lats = [s.xy[1][0] for s in df.geometry]
    return sum(lats)/len(lats)

def find_average_longitude(df):
    longs = [s.xy[0][0] for s in df.geometry]
    return sum(longs)/len(longs)

def max_longitude(df):
    return max(s.xy[0][0] for s in df.geometry)

def max_latitude(df):
    return max(s.xy[1][0] for s in df.geometry)

def min_latitude(df):
    return min(s.xy[1][0] for s in df.geometry)


def fade(value):
    # function for the line width on the chart; fade/decays using a normal distribution type curve 
    mean = -2; std = 0.4; variance = np.square(std)
    return 4*(np.exp(-np.square(value-mean)/2*variance)/(np.sqrt(2*np.pi*variance)))


def colour_fading(street_angle, sunrise_angle):
    a = sunrise_angle - street_angle
    b = sunrise_angle - street_angle - 180
    
    if -7 < a < 3:
        return {'colour':'red', 'linewidth':fade(a), 'alpha':1.0}
    
    if -7 < b < 3:
        return {'colour':'red', 'linewidth':fade(b), 'alpha':1.0} 

    else:
        #colour of the non-highlighted streets
        return {'colour':'grey', 'linewidth':0.7, 'alpha':1} 

def make_date_df(latitude):
    df = pd.DataFrame()
    months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUNE', 'JULY', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
    df['date'] = pd.date_range(start='1/1/2020', end='31/12/2020')
    df['month'] = pd.to_datetime(df['date']).dt.month
    df['day_of_year'] = pd.to_datetime(df['date']).dt.dayofyear
    df['month_name'] = df['month'].apply(lambda x: months[x-1])
    df['sunrise_angle'] = df['day_of_year'].apply(lambda x: sunrise_angle(latitude, x))
    return df

def find_month_position(max_lat, min_lat, d):
    # function to make text move down and then up to signify the rough position of the sunrise
    if max_lat - (2 * d*(max_lat - min_lat)/367) > min_lat:
        month_position = max_lat - (2 * d*(max_lat - min_lat)/367)
    if max_lat - (2 * d*(max_lat - min_lat)/367) < min_lat:
        month_position = min_lat + (2 * (d-183)*(max_lat - min_lat)/367)
    return month_position

    

def make_map(G, df, day_of_year, month, filename, figsize):
    
    lat  = find_average_latitude(df)
    long = find_average_longitude(df)
    max_lat = max_latitude(df)
    min_lat = min_latitude(df)
    max_long = max_longitude(df)
    #month_pos = find_month_position(max_lat, min_lat, day_of_year)

    fig, ax = ox.plot_graph(G, bgcolor='black', axis_off=True, node_size=0, node_color='grey', node_edgecolor='grey', node_zorder=2,
                        edge_color=df['colour'], edge_linewidth=df['linewidth'], edge_alpha=.7, fig_height=figsize, dpi=50, show=False, close=False)
    
    ax.set_title(filename.upper()[:7], fontsize=24, fontweight='bold', color='red', loc='right')
    
    ax.text(max_long-0.01, max_lat-0.001, month, fontsize=18,fontweight='bold', color='red', alpha=1)
    plt.show() 
    return fig.savefig(filename+'.png', facecolor=fig.get_facecolor())




def make_animation(Graph, save_as, figsize, version):
    # create directory
    
    main_dir = ('DIRECTORY NAME')
    png_dir = 'images'
    gif_dir = 'GIFs'
    os.chdir(main_dir)
    if not os.path.exists(main_dir + '/' + save_as):
        os.mkdir(main_dir + '/' + save_as)
    if not os.path.exists(main_dir + save_as + '/' + png_dir):
        os.mkdir(main_dir + save_as + '/' + png_dir)
    if not os.path.exists(main_dir + save_as + '/' + gif_dir):
        os.mkdir(main_dir + save_as + '/' + gif_dir)

    # make master df
    master_df = make_df(Graph)
    
    # find latitude
    lat = find_average_latitude(master_df)
    
    # make date df including sunrise angles
    sunrise_df = make_date_df(lat)
    
    # make maps
    os.chdir('DIRECTORY NAME'+ save_as+'/images')

    
    for index, row in sunrise_df.iterrows():
        sunrise = row['sunrise_angle']
        month = row['month_name']
        day = row['day_of_year']
        df = add_features(master_df, day)
        make_map(Graph, df, day, month, save_as+str(day), figsize)

    # make GIF
    
    os.chdir('DIRECTORY NAME'+ save_as)

    # Create a list of filenames going forwards and backwards
    filenames = [save_as+str(x)+'.png' for x in sunrise_df['day_of_year']]
    print (filenames)


    # Create the frames
    images=[]
    for file_name in filenames:
        file_path = os.path.join(png_dir, file_name)
        #print(file_path)
        file = Image.open(file_path)

        images.append(file)
        file.load()

    # Save into a GIF file that loops forever

    images[0].save(gif_dir + '/' + save_as + str(version) + '.gif', format='GIF',
                   append_images=images[1:],
                   save_all=True, loop=0, duration=100, optimise=True)
    
    return



## San Francisco

In [201]:
SanFran = make_graph('San Francisco, USA', 1)


In [ ]:
make_animation(SanFran, 'San Francisco', 7, version=1)